In [2]:
import os
from onekey_algo import get_param_in_cwd
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

import monai
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt

root = r'D:\x'

In [ ]:
from onekey_algo.custom.components.comp2 import extract, init_from_model, init_from_onekey
from onekey_algo.datasets.image_loader import default_loader
from onekey_algo.custom.components.comp2 import show_cam_on_image
import torch
import os

model, transformer, device = init_from_onekey(rf'D:\x\viz')
#     for n, m in model.named_modules():
#         print('Feature name:', n, "|| Module:", m)
target_layer = "layer4.2.conv3"
gradcam = monai.visualize.GradCAM(nn_module=model, target_layers=target_layer)

save_dir = os.path.join(root, f"viz")
os.makedirs(save_dir, exist_ok=True)
samples = glob(os.path.join(root, 'crop', '*.png'))
for sample in samples:
#     if not os.path.exists(sample) or os.path.exists(rf'D:\2023\viz/{os.path.basename(sample)}'):
#         continue
    img = default_loader(sample)
    sample_ = transformer(img)
    sample_  = sample_.view(1, *sample_.size()).to(device)
    res_cam = gradcam(x=sample_, class_idx=None)
    fig, axes = plt.subplots(1, 2, figsize=(20, 10), facecolor='white')
#     axes[0].imshow(-res_cam[0][0].cpu(), cmap='jet')
    axes[0].imshow(img.resize(sample_.size()[2:]))
    axes[0].axis('off')
#     plt.savefig(f"viz/{os.path.basename(sample).replace('.png', '_se.png')}", bbox_inches = 'tight')
#     plt.show()
#     plt.figure(figsize=(10, 10))
#     plt.axis('off')
    axes[1].imshow(show_cam_on_image(img.resize(sample_.size()[2:]), -res_cam[0][0].cpu(), use_rgb=True, reverse=False))
    axes[1].axis('off')
    plt.savefig(rf'{save_dir}/{os.path.basename(sample)}', bbox_inches = 'tight')
    plt.close()